In [6]:
from result_saver import SaverProvider
from Scratch import metadata_loader
from Scratch import find_closest_calib_jobs

import numpy as np
from tqdm import tqdm
from time import sleep
from datetime import datetime
import json
import os

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
from src import cpp_soft_info as csi

def decode(DEVICE, LOGICAL, XBASIS, ROUNDS, _RESETS, file_name):
    provider = SaverProvider()

    state = "X" if XBASIS else "Z"
    state += LOGICAL
    print(f"state = {state}")


    # Load the metadata
    while True:
        try:
            md = metadata_loader(True, True)
            break
        except:
            sleep(5)
    md = md[md["job_status"] == "JobStatus.DONE"]
    md = md[md["code"] == "RepetitionCodeCircuit"]
    # md = md[md["descr"] == 'subset RepCodes']
    md = md.dropna(subset=["rounds"])
    md = md[md["meas_level"] == 1]
    md['rounds'] = md['rounds'].astype(int)
    md['distance'] = md['distance'].astype(int)

    md = md[md["backend_name"] == DEVICE]
    md = md[md["logical"] == int(LOGICAL)]
    md = md[md["xbasis"] == XBASIS]
    md = md[md["rounds"] == ROUNDS]

    md = md[:1]

    print("shape:", md.shape)

    DOUBLE_MSMT = False # to get also 03-16

    jobs_by_calibration_date = {}
    for index, row in md.iterrows():
        job_id = row['job_id']

        while True:
            try:
                _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
                break
            except:
                sleep(5)

        if calib_creation_date not in jobs_by_calibration_date.keys():
            jobs_by_calibration_date[calib_creation_date] = [job_id]
        else:
            jobs_by_calibration_date[calib_creation_date].append(job_id)

    print(jobs_by_calibration_date)
    print()
    print(f"num of calibrations: {len(jobs_by_calibration_date)}")
    print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")


    d = md["distance"].values[0]

    distances = np.arange(7, d+1, 4)
    distances = distances[::-1]

    
    HANDLE_OUTLIERS = True

    rel_error = 1

    # KDE BANDWIDTHS
    lin = [0.6, 1.2, 1]
    num_points = 20
    # lin = [0.1, 0.7, 1]
    # num_points = 7
    bandwidths = np.linspace(lin[0], lin[1], lin[2])


    for calib_date in jobs_by_calibration_date.keys():
        # get the noise dict of that date
        noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

        # get the KDE of that date

        while True:
            try:
                all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=False)
                break
            except:
                sleep(5)

        kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        while True:
            try:
                all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                                other_date=calib_date, post_process=True,
                                                                                double_msmt=True)
                break
            except:
                sleep(5)
        kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        # Get the mean msmt errors
        p_soft_mean = 0
        p_hard_mean = 0
        for key, value in msmt_err_dict_PS.items():
            p_soft_mean += value['p_soft']
            p_hard_mean += value['p_hard']
        p_soft_mean /= len(msmt_err_dict_PS)
        p_hard_mean /= len(msmt_err_dict_PS)
        print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

        # for i in range(127):
        #     print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
        # print("\nPS", end=' ')
        # for i in range(127):
        #     print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

        # Retrieve the memories
        memories = []
        for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
            d = md[md["job_id"] == job_id]["distance"].values[0] 
            T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

            # Get the job
            job = provider.retrieve_job(job_id)
            memory = job.result().get_memory()
            memories.append(memory)

        # Stack the memories vertically
        big_memory = np.vstack(memories)

        # Get the layout of the last job (same as previous)
        layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
        link_qubits = list(layout_des['link_qubit'].values())
        code_qubits = list(layout_des['code_qubit'].values())

        # Get the pSoft and countMat matrices
        big_layout = link_qubits + code_qubits
        inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

        print(f"Starting to get pSoft and countMat at {datetime.now()}")
        pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                    handleOutliers = HANDLE_OUTLIERS)
        
        print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
        pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                        handleOutliers = HANDLE_OUTLIERS)
        
        print(f"Starting to get pSoftG at {datetime.now()}")
        countMatG, pSoftG = gaussianIQConvertor(big_memory, inverted_q_map, gmm_dict)

        assert pSoft.shape[0] == pSoft_PS.shape[0] == pSoftG.shape[0], "Different number of shots for each decoding method"
        NB_SHOTS = pSoft.shape[0]

        # Subsample decoding
        for D_NEW in tqdm(distances):
            subsets = generate_subsets_with_center(d, D_NEW)

            err_s_G_mean = []
            err_s_G_indiv = []
            err_h_G_indiv = []
            err_h_G_mean = []
            err_s_K_mean = []
            err_s_K_indiv = []
            err_h_K_indiv = []
            err_h_K_mean = []
            err_s_KPS_mean = []
            err_s_KPS_indiv = []
            err_h_KPS_indiv = []
            err_h_KPS_mean = []
            err_h_G_mean_mean = []
            err_h_K_mean_mean = []
            err_h_KPS_mean_mean = []
            err_h_G_mean_mean_50 = []
            err_h_K_mean_mean_50 = []
            err_h_KPS_mean_mean_50 = []

            err_s_RES_K_mean = []
            err_s_RES_K_indiv = []
            for subset in (subsets):             
                cols_to_keep = get_cols_to_keep(subset, T, d)

                # Get the subset of pSoft and countMat
                pSoft_sub = pSoft[:, cols_to_keep]
                pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
                pSoftG_sub = pSoftG[:, cols_to_keep]
                countMat_sub = countMat[:, cols_to_keep]
                countMat_PS_sub = countMat_PS[:, cols_to_keep]
                countMatG_sub = countMatG[:, cols_to_keep]

                # Means for hard decoding
                pSoft_mean = np.mean(pSoft_sub)
                pSoft_PS_mean = np.mean(pSoft_PS_sub)
                pSoftG_mean = np.mean(pSoftG_sub)

                # Get the layout
                layout = get_subsample_layout(subset, link_qubits, code_qubits)

                # Get the noise avgs
                avgs = get_avgs_from_dict(noise_dict, layout)
                noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
                readout = avgs["readout"]
                # noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
                noise_list += [0.4, p_hard_mean, p_soft_mean]
                #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

                # Stim models
                subsampling = (D_NEW != d)

                code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                                msmt_err_dict=msmt_err_dict_PS)
                model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

                code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                                msmt_err_dict=msmt_err_dict_PS)
                model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

                code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

                code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()

                noise_list_soft = noise_list.copy()
                noise_list_soft[5] = 0 # no hard errors
                noise_list_soft[6] = 0 # no soft errors
                code_indiv_for_soft_reset = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list_soft,
                                                                subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                                msmt_err_dict=msmt_err_dict_PS)
                model_indiv_for_soft_reset = code_indiv_for_soft_reset.circuits[LOGICAL].detector_error_model()

                code_mean_for_soft_reset = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list_soft,
                                                                subsampling=subsampling, no_fin_soft=True, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_soft_reset = code_mean_for_soft_reset.circuits[LOGICAL].detector_error_model()
                

                # Decoding

                ### Resets
                res_s_RES_K_mean = csi.decodeConvertorAll(model_mean_for_soft_reset, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_s_RES_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft_reset, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)

                ### Gaussian
                res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
                res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoftG_mean
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                                msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)

                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoftG_mean*1.5
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                                msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_G_mean_mean_50 = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                ### KDE
                res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_mean
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_mean*1.5
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_K_mean_mean_50 = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                
                ### KDE PS
                res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_PS_mean
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                new_noise_list = noise_list.copy()
                new_noise_list[-1] = pSoft_PS_mean*1.5
                code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
                model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
                res_h_KPS_mean_mean_50 = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                            int(LOGICAL), _RESETS, decode_hard=True)
                
                # Append the results
                err_s_G_mean.append(res_s_G_mean.num_errors)
                err_s_G_indiv.append(res_s_G_indiv.num_errors)
                err_h_G_indiv.append(res_h_G_indiv.num_errors)
                err_h_G_mean.append(res_h_G_mean.num_errors)
                err_s_K_mean.append(res_s_K_mean.num_errors)
                err_s_K_indiv.append(res_s_K_indiv.num_errors)
                err_h_K_indiv.append(res_h_K_indiv.num_errors)
                err_h_K_mean.append(res_h_K_mean.num_errors)
                err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
                err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
                err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
                err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

                err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
                err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
                err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)

                err_h_G_mean_mean_50.append(res_h_G_mean_mean_50.num_errors)
                err_h_K_mean_mean_50.append(res_h_K_mean_mean_50.num_errors)
                err_h_KPS_mean_mean_50.append(res_h_KPS_mean_mean_50.num_errors)

                err_s_RES_K_indiv.append(res_s_RES_K_indiv.num_errors)
                err_s_RES_K_mean.append(res_s_RES_K_mean.num_errors)


            # Save the results
            json_Gaussian = {
                "decoding": "gaussian",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_G_mean,
                    "soft_indiv": err_s_G_indiv,
                    "hard_indiv": err_h_G_indiv,
                    "hard_mean": err_h_G_mean,
                    "hard_mean_mean": err_h_G_mean_mean,
                    "hard_mean_mean_50": err_h_G_mean_mean_50,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_G_mean),
                    "soft_indiv": np.mean(err_s_G_indiv),
                    "hard_indiv": np.mean(err_h_G_indiv),
                    "hard_mean": np.mean(err_h_G_mean),
                    "hard_mean_mean": np.mean(err_h_G_mean_mean),
                    "hard_mean_mean_50": np.mean(err_h_G_mean_mean_50),
                },      
                "additional_info": {
                    "job_ids": jobs_by_calibration_date[calib_date],
                    "calib_date": str(calib_date),
                    "noise_list": noise_list,
                    "calib_readout": readout,
                    "p_soft_mean": p_soft_mean,
                    "p_hard_mean": p_hard_mean,
                    "pSoftG_mean": pSoftG_mean,
                    "pSoftG_mean*1.5": pSoftG_mean*1.5,
                },    
            }

            json_KDE = {
                "decoding": "kde",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_K_mean,
                    "soft_indiv": err_s_K_indiv,
                    "hard_indiv": err_h_K_indiv,
                    "hard_mean": err_h_K_mean,
                    "hard_mean_mean": err_h_K_mean_mean,
                    "hard_mean_mean_50": err_h_K_mean_mean_50,
                    "soft_indiv_RES": err_s_RES_K_indiv,
                    "soft_mean_RES": err_s_RES_K_mean,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_K_mean),
                    "soft_indiv": np.mean(err_s_K_indiv),
                    "hard_indiv": np.mean(err_h_K_indiv),
                    "hard_mean": np.mean(err_h_K_mean),
                    "hard_mean_mean": np.mean(err_h_K_mean_mean),
                    "hard_mean_mean_50": np.mean(err_h_K_mean_mean_50),
                    "soft_indiv_RES": np.mean(err_s_RES_K_indiv),
                    "soft_mean_RES": np.mean(err_s_RES_K_mean),
                },   
                "additional_info": {
                    "rel_error": rel_error,
                    "bandwidth_linspace": lin,
                    "num_points_bandwidths": num_points,
                    "pSoft_mean": pSoft_mean,
                    "pSoft_mean*1.5": pSoft_mean*1.5,
                },        
            }

            json_KDE_PS = {
                "decoding": "kde_PS",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_KPS_mean,
                    "soft_indiv": err_s_KPS_indiv,
                    "hard_indiv": err_h_KPS_indiv,
                    "hard_mean": err_h_KPS_mean,
                    "hard_mean_mean": err_h_KPS_mean_mean,
                    "hard_mean_mean_50": err_h_KPS_mean_mean_50,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_KPS_mean),
                    "soft_indiv": np.mean(err_s_KPS_indiv),
                    "hard_indiv": np.mean(err_h_KPS_indiv),
                    "hard_mean": np.mean(err_h_KPS_mean),
                    "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                    "hard_mean_mean_50": np.mean(err_h_KPS_mean_mean_50),
                },   
                "additional_info": {
                    "pSoft_PS_mean": pSoft_PS_mean,
                    "pSoft_PS_mean*1.5": pSoft_PS_mean*1.5,
                },
            }

            if not os.path.exists(file_name):
                data = {}
            else:
                with open(file_name, "r") as f:
                    data = json.load(f)

            if job_id not in data.keys():
                data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
            else:
                data[job_id].append(json_Gaussian)
                data[job_id].append(json_KDE)
                data[job_id].append(json_KDE_PS)
            
            with open(file_name, "w") as f:
                json.dump(data, f, indent=4)

In [7]:
logicals = ['1']
xbasis = [False]
rounds = [50]
devices = ["ibm_kyoto"]

_RESETS = True

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/0.4RESET_{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, _RESETS, file_name)

state = Z1
shape: (1, 18)
{datetime.datetime(2024, 4, 13, 6, 12, 51, 31308, tzinfo=datetime.timezone.utc): ['crd24p2622j0008m9kdg']}

num of calibrations: 1
num of jobs per calibration: [1]
13:18:13 Warning: Idling time set to double 2800.00 ns due to active resets. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 35
13:18:13 Warning: Negative T2 error -21.43 % for qubit 117, setting to 0. T1: 8.683212806704563e-07, T2: 5.299507720130084e-05. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 54
Found jobs for backend ibm_kyoto with closest execution date 2024-04-13 06:27:48.410536+00:00.
Found jobs for backend ibm_kyoto with closest execution date 2024-04-13 06:28:15.289268+00:00.
{0: {'p_hard': 0.6968503937007874, 'p_soft': 0.10699771399542798, 'p_hard_0': 0.6835788671577343, 'p_hard_1': 0.7101219202438405, 'p_soft

Retrieving jobs of 2024-04-13 06:12:51.031308+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.48s/it]


Starting to get pSoft and countMat at 2024-04-16 13:18:44.243910
Starting to get pSoft_PS and countMat_PS at 2024-04-16 13:19:15.229080
Starting to get pSoftG at 2024-04-16 13:19:23.858152


100%|██████████| 11/11 [06:41<00:00, 36.50s/it]
